In [1]:
pip install sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sqlalchemy
import pymysql

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:db@localhost/db')

engine.connect()
print("Connected Successfully!")

Connected Successfully!


In [4]:
import pandas as pd

pd.read_sql('SHOW TABLES',engine)

,Tables_in_pokedb
0,metrics_table
1,pokemon_table
2,pokemon_types
3,stats_table
4,types_table


In [14]:
pd.read_sql('DESCRIBE pokemon_table;',engine)

,Field,Type,Null,Key,Default,Extra
0,pokemon_id,int,NO,PRI,None,
1,pokemon_name,varchar(50),NO,,None,
2,generation,varchar(10),NO,,None,
3,is_legendary,tinyint(1),NO,,None,
4,is_pseudolegend,tinyint(1),NO,,None,


In [5]:
pd.read_sql("""
SELECT 
    TABLE_NAME,
    COLUMN_NAME,
    CONSTRAINT_NAME,
    REFERENCED_TABLE_NAME,
    REFERENCED_COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE TABLE_SCHEMA = 'pokedb'
AND REFERENCED_TABLE_NAME IS NOT NULL;
""", engine)

,TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME,REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
0,metrics_table,pokemon_id,metrics_table_ibfk_1,pokemon_table,pokemon_id
1,pokemon_types,pokemon_id,pokemon_types_ibfk_1,pokemon_table,pokemon_id
2,pokemon_types,type_id,pokemon_types_ibfk_2,types_table,type_id
3,stats_table,pokemon_id,stats_table_ibfk_1,pokemon_table,pokemon_id


In [15]:
pd.read_sql('SELECT * FROM pokemon_table where pokemon_id=9999',engine)

,pokemon_id,pokemon_name,generation,is_legendary,is_pseudolegend


In [ ]:
from sqlalchemy import text

# with engine.begin() as conn:
#     conn.execute(text("""
#     INSERT INTO pokemon_table (pokemon_id,pokemon_name,generation,is_legendary,is_pseudolegend)
#     VALUES (9999,'Testerman','Gen-99',False,False);
# """))
    
# print("Inserted Test Pokemon Successfully!")

Inserted Test Pokemon Successfully!


In [ ]:
# with engine.begin() as conn:
#     conn.execute(text("""
#     INSERT INTO stats_table (pokemon_id,hp,attack,defense,sp_atk,sp_def,speed)
#     VALUES (9999,99,99,99,99,99,99);
# """))
    
# print("Inserted Test Pokemon Successfully!")

Inserted Test Pokemon Successfully!


In [14]:
with engine.begin() as conn:
    conn.execute(text("""
    DELETE from stats_table where pokemon_id=9999;
"""))

In [17]:
data = pd.read_csv('required_pokedataset.csv')
data.head()

,id,Name,HP,Attack,Defense,Sp.Atk,Sp.Def,Speed,Type_1,Type_2,Is_Legendary,Generation,Is_Pseudo_Legendary,Total_Stats,Offensive_Score,Defensive_Score,Specialization_Score
0,1,Bulbasaur,45,49,49,65,65,45,Grass,Poison,False,Gen-1,False,318,114,159,9.465728
1,2,Ivysaur,60,62,63,80,80,60,Grass,Poison,False,Gen-1,False,405,142,203,9.751923
2,3,Venusaur,80,82,83,100,100,80,Grass,Poison,False,Gen-1,False,525,182,263,9.751923
3,4,Charmander,39,52,43,60,50,65,Fire,NaN,False,Gen-1,False,309,112,132,9.853933
4,5,Charmeleon,58,64,58,80,65,80,Fire,NaN,False,Gen-1,False,405,144,181,10.114346


In [20]:
data['id'].nunique(),len(data)

(1025, 1025)

In [21]:
pokemon_data = data[['id','Name','Generation','Is_Legendary','Is_Pseudo_Legendary']].copy()
pokemon_data

,id,Name,Generation,Is_Legendary,Is_Pseudo_Legendary
0,1,Bulbasaur,Gen-1,False,False
1,2,Ivysaur,Gen-1,False,False
2,3,Venusaur,Gen-1,False,False
3,4,Charmander,Gen-1,False,False
4,5,Charmeleon,Gen-1,False,False
...,...,...,...,...,...
1020,1021,Raging-bolt,Gen-9,False,False
1021,1022,Iron-boulder,Gen-9,False,False
1022,1023,Iron-crown,Gen-9,False,False
1023,1024,Terapagos,Gen-9,True,False


In [22]:
pokemon_data.columns = ['pokemon_id','pokemon_name','generation','is_legendary','is_pseudolegend']

In [37]:
pokemon_data

,pokemon_id,pokemon_name,generation,is_legendary,is_pseudolegend
0,1,Bulbasaur,Gen-1,False,False
1,2,Ivysaur,Gen-1,False,False
2,3,Venusaur,Gen-1,False,False
3,4,Charmander,Gen-1,False,False
4,5,Charmeleon,Gen-1,False,False
...,...,...,...,...,...
1020,1021,Raging-bolt,Gen-9,False,False
1021,1022,Iron-boulder,Gen-9,False,False
1022,1023,Iron-crown,Gen-9,False,False
1023,1024,Terapagos,Gen-9,True,False


In [23]:
pokemon_stats = data[['id','HP','Attack','Defense','Sp.Atk','Sp.Def','Speed']].copy()
pokemon_stats.columns = ['pokemon_id','hp','attack','defense','sp_atk','sp_def','speed']
pokemon_stats

,pokemon_id,hp,attack,defense,sp_atk,sp_def,speed
0,1,45,49,49,65,65,45
1,2,60,62,63,80,80,60
2,3,80,82,83,100,100,80
3,4,39,52,43,60,50,65
4,5,58,64,58,80,65,80
...,...,...,...,...,...,...,...
1020,1021,125,73,91,137,89,75
1021,1022,90,120,80,68,108,124
1022,1023,90,72,100,122,108,98
1023,1024,90,65,85,65,85,60


In [24]:
pokemon_metrics = data[['id','Total_Stats','Offensive_Score','Defensive_Score','Specialization_Score']].copy()
pokemon_metrics.columns = ['pokemon_id','total_stats','offensive_score','defensive_score','specialization_score']
pokemon_metrics

,pokemon_id,total_stats,offensive_score,defensive_score,specialization_score
0,1,318,114,159,9.465728
1,2,405,142,203,9.751923
2,3,525,182,263,9.751923
3,4,309,112,132,9.853933
4,5,405,144,181,10.114346
...,...,...,...,...,...
1020,1021,590,210,305,26.583203
1021,1022,590,188,278,22.571368
1022,1023,590,194,298,16.848343
1023,1024,450,130,260,13.038405


In [42]:
all_types = pd.concat([data['Type_1'],data['Type_2']]).dropna().unique()
all_types

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [43]:
pokemon_types = pd.DataFrame({"type_name":all_types})
pokemon_types

,type_name
0,Grass
1,Fire
2,Water
3,Bug
4,Normal
5,Poison
6,Electric
7,Ground
8,Fairy
9,Fighting


In [44]:
data['id'].duplicated().sum()

np.int64(0)

In [45]:
pokemon_data.to_sql(
    "pokemon_table",
    con=engine,
    if_exists='append',
    index=False
)

print("Pokemon Data Inserted Successfully")

Pokemon Data Inserted Successfully


In [46]:
pd.read_sql('SELECT * FROM pokemon_table',engine)

,pokemon_id,pokemon_name,generation,is_legendary,is_pseudolegend
0,1,Bulbasaur,Gen-1,0,0
1,2,Ivysaur,Gen-1,0,0
2,3,Venusaur,Gen-1,0,0
3,4,Charmander,Gen-1,0,0
4,5,Charmeleon,Gen-1,0,0
...,...,...,...,...,...
1020,1021,Raging-bolt,Gen-9,0,0
1021,1022,Iron-boulder,Gen-9,0,0
1022,1023,Iron-crown,Gen-9,0,0
1023,1024,Terapagos,Gen-9,1,0


In [47]:
pokemon_stats.to_sql(
    "stats_table",
    con=engine,
    if_exists='append',
    index=False
)
print("Pokemon Stats Inserted Successfully!")

Pokemon Stats Inserted Successfully!


In [49]:
pd.read_sql('SELECT * FROM stats_table',engine)

,pokemon_id,hp,attack,defense,sp_atk,sp_def,speed
0,1,45,49,49,65,65,45
1,2,60,62,63,80,80,60
2,3,80,82,83,100,100,80
3,4,39,52,43,60,50,65
4,5,58,64,58,80,65,80
...,...,...,...,...,...,...,...
1020,1021,125,73,91,137,89,75
1021,1022,90,120,80,68,108,124
1022,1023,90,72,100,122,108,98
1023,1024,90,65,85,65,85,60


In [50]:
pokemon_metrics.to_sql(
    "metrics_table",
    con=engine,
    if_exists='append',
    index=False
)
print("Pokemon Metrics Inserted Successfully!")

Pokemon Metrics Inserted Successfully!


In [51]:
pokemon_types.to_sql(
    "types_table",
    con=engine,
    if_exists='append',
    index=False
)
print("Pokemon Types Inserted Successfully!")

Pokemon Types Inserted Successfully!


In [54]:
type_lookup = pd.read_sql('SELECT * FROM types_table',engine)
type_dict = dict(zip(type_lookup['type_name'],type_lookup['type_id']))
type_dict

{'Bug': 4,
 'Dark': 16,
 'Dragon': 15,
 'Electric': 7,
 'Fairy': 9,
 'Fighting': 10,
 'Fire': 2,
 'Flying': 18,
 'Ghost': 13,
 'Grass': 1,
 'Ground': 8,
 'Ice': 14,
 'Normal': 5,
 'Poison': 6,
 'Psychic': 11,
 'Rock': 12,
 'Steel': 17,
 'Water': 3}

In [55]:
rows = []

for _,row in data.iterrows():
    rows.append({
        "pokemon_id":row['id'],
        "type_id":type_dict[row['Type_1']]
    })

    if pd.notna(row['Type_2']):
        rows.append({
            "pokemon_id":row['id'],
            "type_id":type_dict[row['Type_2']]
        })

pokemon_types = pd.DataFrame(rows)

In [68]:
pokemon_types.head(10)

,pokemon_id,type_id
0,1,1
1,1,6
2,2,1
3,2,6
4,3,1
5,3,6
6,4,2
7,5,2
8,6,2
9,6,18


In [69]:
pokemon_types.to_sql(
    "pokemon_types",
    con=engine,
    if_exists='append',
    index=False
)

print("Pokemon Types Inserted Successfully!")

Pokemon Types Inserted Successfully!


In [31]:
pd.read_sql('SELECT pokemon_name,count(pokemon_name) as pokecount FROM pokemon_table GROUP BY pokemon_name having pokecount>1;',engine)

,pokemon_name,pokecount


In [41]:
pd.read_sql('SELECT pokemon_id,count(pokemon_id) FROM stats_table group by pokemon_id having count(pokemon_id)>1;',engine)

,pokemon_id,count(pokemon_id)


In [43]:
pd.read_sql('SELECT * from metrics_table;',engine)

,pokemon_id,total_stats,offensive_score,defensive_score,specialization_score
0,1,318,114,159,9.46573
1,2,405,142,203,9.75192
2,3,525,182,263,9.75192
3,4,309,112,132,9.85393
4,5,405,144,181,10.11430
...,...,...,...,...,...
1020,1021,590,210,305,26.58320
1021,1022,590,188,278,22.57140
1022,1023,590,194,298,16.84830
1023,1024,450,130,260,13.03840


In [46]:
pd.read_sql('SELECT * from pokemon_types where type_id is null',engine)

,pokemon_id,type_id


In [49]:
pd.read_sql('SELECT type_id,count(type_id) from types_table group by type_id having count(type_id)>1;',engine)

,type_id,count(type_id)


In [53]:
pd.read_sql('SELECT * FROM pokemon_table join stats_table on pokemon_table.pokemon_id=stats_table.pokemon_id join metrics_table on stats_table.pokemon_id=metrics_table.pokemon_id join pokemon_types on pokemon_table.pokemon_id=pokemon_types.pokemon_id where pokemon_name="Charmander"',engine)

,pokemon_id,pokemon_name,generation,is_legendary,is_pseudolegend,pokemon_id,hp,attack,defense,sp_atk,sp_def,speed,pokemon_id,total_stats,offensive_score,defensive_score,specialization_score,pokemon_id,type_id
0,4,Charmander,Gen-1,0,0,4,39,52,43,60,50,65,4,309,112,132,9.85393,4,2
